# Spam classifer with Natural Language processing and Naive Bayes

## Libaries importing

In [14]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

from io import BytesIO
import requests
import tarfile

## Importing datasets

In [12]:
spam_ham_sms = pd.read_csv("datasets/sms(ham-spam).csv").drop("Unnamed: 0", axis=1)
spam_ham_sms.sample(3)

,labels,messages
2041,ham,You always make things bigger than they are
3287,ham,Not to worry. I'm sure you'll get it.
2352,spam,Download as many ringtones as u like no restri...


In [24]:
def extract_spam_data():
    Base_url = "https://spamassassin.apache.org/old/publiccorpus"

    files = ["20021010_easy_ham.tar.bz2",
             "20021010_hard_ham.tar.bz2",            
             "20021010_spam.tar.bz2"]

    Output_dir = "datasets/emailSpam"

    for filename in files:
        content = requests.get(f"{Base_url}/{filename}").content
        fin = BytesIO(content)

        with tarfile.open(fileobj=fin, mode='r:bz2') as tf:
            tf.extractall(Output_dir)

    return files

In [25]:
files = extract_spam_data()